In [3]:
%%configure -f
{"driverMemory": "4G", "driverCores": 2, "executorMemory": "12G", "executorCores": 6, "numExecutors": 3}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,None,pyspark,idle,,,None,✔


In [6]:
from typing import List

from pyspark import SparkFiles
from subprocess import call
import sys


def install_deps(deps: List[str]) -> None:
    call([sys.executable, '-m', 'pip', 'install', '-q', '-t', SparkFiles.getRootDirectory(), *deps])


install_deps(['numpy', 'matplotlib', 'pandas', 'scipy', 'seaborn'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from typing import Optional
import json
from pyspark.sql import SparkSession, DataFrame

spark: SparkSession

def load_distance_calculation_df(aggregation_pipeline: Optional[List] = None) -> DataFrame:
    if aggregation_pipeline is None:
        aggregation_pipeline = []
    
    return (
        spark
        .read
        .format("mongodb")
        .option("partitioner", "com.mongodb.spark.sql.connector.read.partitioner.SamplePartitioner")
        .option("partitioner.options.partition.field", "_id")
        .option("partitioner.options.partition.size", "64")
        .option("partitioner.options.samples.per.partition", "2")
        .option("database", "enhancer3d")
        .option("collection", "distance_calculation")
        .option("aggregation.pipeline", json.dumps(aggregation_pipeline))
        .load()
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…